In [2]:
import pickle

In [3]:
import numpy as np

from lib.envs.gridworld import GridworldEnv # 상위 폴더내의 lib 폴더 -> env 폴더 -> gridworld.py 파일에서 GridwordlEnv 를 가져옴

In [4]:
env = GridworldEnv() # 강화학습을 위한 환경을 env 변수에 넣음

### Evaluation Function

In [6]:
def policy_eval(random_policy, env, discount_factor=1.0, theta=0.00001): # theta 는 interation 을 멈추는 시기를 알려주는 역치
    # 각 state 의 value 값을 0으로 초기화
    V = np.zeros(env.nS)
    
    while True:
        # 이전 iteration 에 비해 State Value 값이 얼마나 값이 변했는지 확인하기 위해 설정
        delta = 0  
        
        for state in range(env.nS):
            # env.nS = 16 이므로 state 의 값 0~15 까지 => 모든 state 를 한 번씩 돌게된다
            v = 0 # 초기화
            
            # random uniform policy (0.25로 통일) 를 action 별로 action probability 와 함께 가지고 온다
            for action, action_prob in enumerate(random_policy[state]):
                
                # 반드시 기억해야 할 것, 
                # env.P는 {action: [(probability, nextstate, reward, done)]} 형식으로 정의 되어 있다.
                # env.P[state][action] = 특정 state 에서 특정 actiond에 대한 P, S', R, Done 의 값
                # V[next_state] = 다음 state 의 value
                
                # for ~~~~ in env.P[state][action]: 이므로,
                # 현재 k 에 대해 특정 state 에 대해서 모든 action 별로 하나씩 value 값을 업데이트 한다
                for  transition_prob, next_state, reward, done in env.P[state][action]:
                    # Value 값 계산, 교재 246 페이지 참고
                    
                    # for 문을 이용해서 각 state 마다 서로 다른 action 값에 따른 next state 의 value 값을
                    # 계속 더해주기 위해 (Sigma 를 구현하기 위해) += 을 사용해주도록 한다 
                    
                    v += action_prob * (reward + discount_factor * transition_prob * V[next_state])
            
            # 이전 iteration 에 비해 State Value 값이 얼마나 값이 변했는지 확인 = delta
            delta = max(delta, np.abs(v - V[state]))
            # State 의 value 값 update
            V[state] = v
        # value 의 변화량이 theta (정해준 역치)보다 적으면 break 
        if delta < theta:
            break
            
    return np.array(V)

policy_eval 함수는 state돌면서, `state value function`을 새로운 값으로 업데이트 해주는 함수. 

### Imporovement Function

In [8]:
env.nA

4

In [12]:
np.eye(env.nA)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [16]:
np.zeros(env.nA)

array([0., 0., 0., 0.])

In [14]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    
    # Q-value (action_value) 계산하는 함수 정의 
    def one_step_lookahead(state, State_Value): 
        # 이 함수는, state랑 state_value(모든 state 값 통째로) 받아서 해당 state의 각 액션별 Q값을 계산해서 리턴해 준다. 
        action_value = np.zeros(env.nA) # 초기화 시켜줌 : array([0., 0., 0., 0.]) & shape = (4,)
        
        for action in range(env.nA): # 각 action 별로 value 구하기 (후에 state 와 같이 for문을 돌게 된다)
            
            for transition_prob, next_state, reward, done in env.P[state][action]: # next state 에서 value 값을 구하기위해
                # 교재 240 page 공식 참조
                # State_Value[next_state] 는 Next state 에서 V pi 를 나타낸다
                # State_Value 가 Evaluate 된 Value 이므로 가장 좋은 policy를 나타내게 될 것이다
                action_value[action] += reward + (discount_factor * transition_prob * State_Value[next_state])
                
                # print("action_value (Q-value):",action_value)
                
        return action_value
    
    # Start with a random policy
    # 최초의 policy 는 state 마다 0.25 로 모두 초기화 (동 서 남 북 전부 0.25의 확률로 움직인다)
    
    policy = np.ones([env.nS, env.nA]) / env.nA
    '''
    policy ↓
     ([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])'''
    
    # improvement 가 한 번 끝난 이후에는 policy 를 improve 한다
    
    while True:
        
        # 1. 가장먼저 해야 할 일 : 현재 policy 를 evaluation        
        state_value = policy_eval_fn(policy, env, discount_factor) # <- policy evaluation code
        print("==============================================")
        print("State_value", )
        print(state_value.reshape([4,4]))

        # 각 State 별로 policy 가 evaluation 되었다 (state 별로 return 을 얼마 받을지 예측)
        
        # 2. 이제부터는 improvement 를 해야한다
        
        # while 문 종료 시점
        policy_stable = True
        
        # 각각의 state 에 대하여 계산
        for state in range(env.nS): # state 0부터 15까지 계산
            print("\n")
            print("==============================================")
            print("시작 state =",state)
            
            # policy[state] 이므로 state 별로 돌아가며 최적의 policy 를 구한다
            # argmax 함수를 이용해서 최적의 policy 를 구하도록 한다
            # argmax 는 max 값을 가지고 있는 원소의 'INDEX' 를 반환한다 [0,0,1,0] 이라면 2 를 반환
            # 즉 chosen_action 에는 해당 state 의 policy 중 가장 높은 값을 가지고 있는 action의 인덱스를 반환시켜주는 것이다
            
            # state 마다 할당되어 있는 policy 를 argmax 해서 가장 알맞은 action 값을 가지고 온다
            # 하지만 처음의 policy 는 random 한 policy 이므로 옳지 않은 policy 가 저장되어 있을 것이다
            # 이후에 나오는 policy_eval 를 이용하여 value 값을 다시 구한 뒤, policy 를 구해본다.
            chosen_action = np.argmax(policy[state])
            print("chosen_action", chosen_action ,"\n")
            print("before improve policy", policy[state])
            
            # policy_eval 함수를 이용해 state 별로 action 의 value 값을 계산
            # state 1 에서 action 0 의 value 는 몇인지, action 1 의 value 는 몇인지 등등을 쭉 구한다
            action_values = one_step_lookahead(state, state_value)
            print("state {} 에서의 최종 action value (Q-value) 값:{}".format(state,action_values), "\n")
            # action_value 가 return 되었다
            
            # 계산한 것을 토대로 그때그때의 best action 을 찾는다
            best_action = np.argmax(action_values)
            print("가장 큰 value 값:", action_values[best_action])
            print("best_action",best_action)
            
            # Greedily update the policy
            # 두개가 같다면 True 값을 반환
            if chosen_action != best_action:
                policy_stable = False
                print("기존의 action (chosen_action) 과 improvement 한 action (best_action) 이 서로 다르다 \n")
            elif chosen_action == best_action:
                print("기존의 action (chosen_action) 과 improvement 한 action (best_action) 이 서로 같다 \n")
            
            policy[state] = np.eye(env.nA)[best_action] # np.eye = 단위행렬 생성, best action 하나를 선택해서 index 로 주면
                                                        # 그에 해당하는 policy 가 만들어진다
                                                        # 예를들어 array([1, 0, 0, 0]) 이런식으로 만들어진다
                                                        # 이러한 코드를 이용해서 해당 state 에 맞는 policy 를 주도록한다.
            '''
            np.eye(env.nA) ↓
           ([[1., 0., 0., 0.],
             [0., 1., 0., 0.],
             [0., 0., 1., 0.],
             [0., 0., 0., 1.]]) 여기서 index 를 1 로 주면 [0, 1, 0, 0] 이 튀어 나올 것
             '''
            print("after improve policy", policy[state])
        print("")
        
        # State 별로 다 돌면 policy_state 를 검사한다
        # If the policy is stable we've found an optimal policy. Return it
        # chosen_action 와 best_action 이 같지 않다면 False 이므로 반환이 되지 않으므로 while 문이 종료되지 않는다
        # chosen_action 와 best_action 이 같다면 True 가 되고 return 값이 생기므로 while 문이 종료된다
        
        # ☆모든 state 에 대해서 전부 같아야한다☆
        print("policy_stable?", policy_stable)
        if policy_stable:
            print("Policy 가 stable 하다")
            return policy, state_value
        else:
            print("Policy 가 stable 하지 않다")
            print("==============================================")

In [15]:
policy, v = policy_improvement(env)

State_value
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]


시작 state = 0
chosen_action 0 

before improve policy [0.25 0.25 0.25 0.25]
state 0 에서의 최종 action value (Q-value) 값:[0. 0. 0. 0.] 

가장 큰 value 값: 0.0
best_action 0
기존의 action (chosen_action) 과 improvement 한 action (best_action) 이 서로 같다 

after improve policy [1. 0. 0. 0.]


시작 state = 1
chosen_action 0 

before improve policy [0.25 0.25 0.25 0.25]
state 1 에서의 최종 action value (Q-value) 값:[-14.99993529 -20.99990698 -18.9999206   -1.        ] 

가장 큰 value 값: -1.0
best_action 3
기존의 action (chosen_action) 과 improvement 한 action (best_action) 이 서로 다르다 

after improve policy [0. 0. 0. 1.]


시작 state = 2
chosen_action 0 

before improve policy [0.25 0.25 0.25 0.25]
state 2 에서의 최종 action value (Q-value) 값:[-20.99990698 -22.99989761 -20.99991379 -14.99993529] 

가장 큰

In [17]:
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]



In [19]:
print("Value Function:")
print(v)
print("")

# optimal policy 를 따라 갔을때의 value 값
print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

